In [29]:

def generate_state_equ1(mass_matrix, forcing_vector, qdot, qdd, u):
    '''
    given mass_matrix and forcing_vector of a Kane's Method it
    generates state equation :
                         xdot=f(x)+g(x).u
               
                  'ATTENTION : it assumes qdd0 as an Input u'
    
    '''
  
    xdot = mass_matrix.inv()*forcing_vector
    fx=sm.Matrix([xdot[i] for i in range(2*len(qdot))]) 
    fx[len(qdot)]=0
    gx =sm.zeros(4,1)
    gx[len(qdot)]=1
    gx[len(qdot)+1]= xdot[len(qdot)]


    return fx, gx



In [30]:
# coding: utf-8
'''
-----------
To Do :
  - parameter shoul be read from a file isntead of dictionary!
  
'''
#=============================================================
# Standard Python modules
#=============================================================

#=============================================================
# External Python modules
#=============================================================
#from __future__ import division, print_function
from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=True)

import sympy as sm
import sympy.physics.mechanics as me
import numpy as np
import mpmath as mp
import scipy as sc

#=============================================================
# Standard Python modules
#=============================================================
from functions import *

#=============================================================
# Systme Model
#=============================================================

# Defining symbolic Variables

n = 1
q = me.dynamicsymbols('q:{}'.format(n + 1))  # generalized coordinates
qdot = me.dynamicsymbols('qdot:{}'.format(n + 1))  #generalized speeds
qdd = me.dynamicsymbols('qddot:{}'.format(n + 1))
f = me.dynamicsymbols('f')
u = sm.symbols('u')
m = sm.symbols('m:{}'.format(n + 1))
J = sm.symbols('J:{}'.format(n + 1))
l = sm.symbols('l1:{}'.format(n+1))  # lenght of each pendlum
a = sm.symbols('a1:{}'.format(n+1))  #location of Mass-centers
d = sm.symbols('d1:{}'.format(n + 1))  #viscous damping coef.
g, t = sm.symbols('g t')

# intertial reference frame
In_frame = me.ReferenceFrame('In_frame')

# Origninal Point O in Reference Frame :
O = me.Point('O')
O.set_vel(In_frame, 0)

# The Center of Mass Point on cart :
C0 = me.Point('C0')
C0.set_pos(O, q[0] * In_frame.x)
C0.set_vel(In_frame, qdot[0] * In_frame.x)

cart_inertia_dyadic = me.inertia(In_frame, 0, 0, J[0])
cart_central_inertia = (cart_inertia_dyadic, C0)

cart = me.RigidBody('Cart', C0, In_frame, m[0], cart_central_inertia)

kindiffs = [q[0].diff(t) - qdot[0]]  # entforcing qdot=Omega

frames = [In_frame]
mass_centers = [C0]
joint_centers = [C0]
central_inertias = [cart_central_inertia]
forces = [(C0, f * In_frame.x - m[0] * g * In_frame.y)]
torques = []

# cart_potential = 1 / 2 * d[0] * qdot[1]**2
# potentials = [cart_potential]
# cart.potential_energy= cart_potential

rigid_bodies = [cart]
# Lagrangian0 = me.Lagrangian(In_frame, rigid_bodies[0])
# Lagrangians=[Lagrangian0]

for i in range(n):
    #Creating new reference frame
    Li = In_frame.orientnew('L' + str(i), 'Axis',
                            [sm.pi / 2 - q[i + 1], In_frame.z])
    Li.set_ang_vel(In_frame, -qdot[i + 1] * In_frame.z)
    frames.append(Li)

    # Creating new Points representing mass_centers
    Pi = mass_centers[-1].locatenew('a' + str(i + 1), a[i] * Li.x)
    Pi.v2pt_theory(joint_centers[-1], In_frame, Li)
    mass_centers.append(Pi)

    #Creating new Points representing Joints
    Jointi = joint_centers[-1].locatenew('jont' + str(i + 1), l[i] * Li.x)
    Jointi.v2pt_theory(joint_centers[-1], In_frame, Li)
    joint_centers.append(Jointi)

    #adding forces
    forces.append((Pi, -m[i + 1] * g * In_frame.y))

    #adding torques
    if i == 0:
        torqueVectori = (-d[0] * qdot[1]) * frames[1].z
        torques.append((Li, torqueVectori))

    else:
        torqueVectori = -d[i] * (qdot[i + 1] - qdot[i]) * In_frame.z
        torques.append((Li, torqueVectori))

    #adding cential inertias
    IDi = me.inertia(frames[i + 1], 0, 0, J[i + 1])
    ICi = (IDi, mass_centers[i + 1])
    central_inertias.append(ICi)

    LBodyi = me.RigidBody('L' + str(i + 1) + '_Body', mass_centers[i + 1],
                          frames[i + 1], m[i + 1], central_inertias[i + 1])
    rigid_bodies.append(LBodyi)

    kindiffs.append(q[i + 1].diff(t) - qdot[i + 1])

#generalized force
loads = torques + forces

#Kane's Method --> Equation of motion
Kane = me.KanesMethod(In_frame, q, qdot, kd_eqs=kindiffs)
fr, frstar = Kane.kanes_equations(rigid_bodies, loads)

mass_matrix = sm.trigsimp(Kane.mass_matrix_full)
forcing_vector = sm.trigsimp(Kane.forcing_full)


print(
    'mass_matrix and forcing_vector are calculated! \n'
)
#xdot_expr=(mass_matrix.inv()*forcing_vector)


# defining parameter values according to number of pendulum n :
if n == 1:
    param_values = config.parameter_values_simple_pendulum

elif n == 2:
    param_values = config.parameter_values_double_pendulum

elif n == 3:
    param_values = config.parameter_values_triple_pendulum

param_symb = list( l + a + m  + J + d + (g, f))
param_list = zip(param_symb, param_values)
param_dict = dict(param_list)


# substituting parameters to mass and forcing_vector
mass_matrix_simplified= mass_matrix.subs(param_dict).simplify()
forcing_vector_simplified= forcing_vector.subs(param_dict).simplify()
# finding fx and gx wiht qdd0 as input
fx, gx = generate_state_equ1(mass_matrix_simplified, forcing_vector_simplified, qdot, qdd, u)
#config.fx_expr = (fx)
#config.gx_expr = (gx)

print(
    'system model succesfully finished ! \n fx_expr and gx_expr are stored in config.py'
)

mass_matrix and forcing_vector are calculated! 

system model succesfully finished ! 
 fx_expr and gx_expr are stored in config.py


In [ ]:
mass_matrix

In [ ]:
forcing_vector

In [ ]:
mass_matrix_simplified

In [ ]:
forcing_vector_simplified

In [ ]:
mass_matrix_simplified*sm.Matrix(qdot+ qdd)

In [ ]:
fx

In [ ]:
gx

In [3]:
sol = mass_matrix_simplified.inv()*forcing_vector_simplified

In [ ]:
f

In [4]:
fx2, gx2 = generate_state_equ(mass_matrix_simplified, forcing_vector_simplified, qdot, qdd, u)

In [5]:
(fx2+gx2*u)

Matrix([
[                                                                           qdot0],
[                                                                           qdot1],
[                                                                               u],
[-3.16149160600208*u*cos(q1) + 0.185707918585649*qdot1 + 31.0142326548804*sin(q1)]])

In [6]:
fx+gx*u

Matrix([
[                                                                           qdot0],
[                                                                           qdot1],
[                                                                               u],
[-3.16149160600208*u*cos(q1) + 0.185707918585649*qdot1 + 31.0142326548804*sin(q1)]])

In [11]:
fx+gx*u

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢⎛  ⎛                                      ⎛                    2             
⎢⎝- ⎝(0.107676⋅cos(q₁) - 0.233298⋅cos(q₂))⋅⎝- 0.209192805376⋅cos (q₁) + 0.7415
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                   

In [14]:
sm.zeros(3,1)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [17]:
xdot = mass_matrix.inv()*forcing_vector
fxx=sm.Matrix([xdot[i] for i in range(2*len(qdot))]) 

In [20]:
fxx

⎡                                             q̇₀                             
⎢                                                                             
⎢                                             q̇₁                             
⎢                                                                             
⎢                                                    ⎛       2   ⎞ ⎛         2
⎢    a₁⋅m₁⋅(a₁⋅g⋅m₁⋅sin(q₁) + d₁⋅q̇₁)⋅cos(q₁)        ⎝J₁ + a₁ ⋅m₁⎠⋅⎝a₁⋅m₁⋅q̇₁ 
⎢- ──────────────────────────────────────────── + ────────────────────────────
⎢      2   2    2       ⎛       2   ⎞                 2   2    2       ⎛      
⎢  - a₁ ⋅m₁ ⋅cos (q₁) + ⎝J₁ + a₁ ⋅m₁⎠⋅(m₀ + m₁)   - a₁ ⋅m₁ ⋅cos (q₁) + ⎝J₁ + a
⎢                                                                             
⎢           ⎛         2            ⎞                                          
⎢     a₁⋅m₁⋅⎝a₁⋅m₁⋅q̇₁ ⋅sin(q₁) + f⎠⋅cos(q₁)          (m₀ + m₁)⋅(a₁⋅g⋅m₁⋅sin(q
⎢- ──────────────────────────────────────────── + ──

In [21]:
fxx[2]=0

In [22]:
fxx

⎡                                             q̇₀                             
⎢                                                                             
⎢                                             q̇₁                             
⎢                                                                             
⎢                                              0                              
⎢                                                                             
⎢           ⎛         2            ⎞                                          
⎢     a₁⋅m₁⋅⎝a₁⋅m₁⋅q̇₁ ⋅sin(q₁) + f⎠⋅cos(q₁)          (m₀ + m₁)⋅(a₁⋅g⋅m₁⋅sin(q
⎢- ──────────────────────────────────────────── + ────────────────────────────
⎢      2   2    2       ⎛       2   ⎞                 2   2    2       ⎛      
⎣  - a₁ ⋅m₁ ⋅cos (q₁) + ⎝J₁ + a₁ ⋅m₁⎠⋅(m₀ + m₁)   - a₁ ⋅m₁ ⋅cos (q₁) + ⎝J₁ + a

                ⎤
                ⎥
                ⎥
                ⎥
                ⎥
                ⎥
                ⎥
₁) +

In [31]:
fx

⎡                                      q̇₀                                    
⎢                                                                             
⎢                                      q̇₁                                    
⎢                                                                             
⎢                                       0                                     
⎢                                                                             
⎢                                                           2                 
⎢4.1912⋅(0.01⋅q̇₁ + 1.6700544⋅sin(q₁))      0.0289816576⋅q̇₁ ⋅sin(q₁)⋅cos(q₁) 
⎢────────────────────────────────────── - ────────────────────────────────────
⎢                  2                                        2                 
⎣- 0.0289816576⋅cos (q₁) + 0.2256877376   - 0.0289816576⋅cos (q₁) + 0.22568773

  ⎤
  ⎥
  ⎥
  ⎥
  ⎥
  ⎥
  ⎥
  ⎥
──⎥
  ⎥
76⎦

In [32]:
gx

⎡                                                  0                          
⎢                                                                             
⎢                                                  0                          
⎢                                                                             
⎢                                                  1                          
⎢                                                                             
⎢                                                                             
⎢  0.713509888⋅(0.01⋅q̇₁ + 1.6700544⋅sin(q₁))⋅cos(q₁)           0.038421080449
⎢- ────────────────────────────────────────────────── + ──────────────────────
⎢                          2                                                  
⎣    - 0.12146792333312⋅cos (q₁) + 0.94590244582912     - 0.12146792333312⋅cos

                        ⎤
                        ⎥
                        ⎥
                        ⎥
                        ⎥
